# Bangla Blog Post Classifier

In [1]:
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
from pprint import pprint
from tqdm import tqdm
df = pd.read_json('./choturmatrik.json', encoding='utf-8')
print(df.head())

                                              content        tags
1   অফিসে তেমন ব্যস্ততা নেই, \nকেমন একটা নিঃসঙ্গ ভ...       কবিতা
8   সুখ আমি কল্পনা করব না \nআমার তো নেই সুখ এই প্র...       কবিতা
10  আমাদের ফেসবুক ঠিকানা: \nগ্রুপ: Facebook.com/gr...  আবোল তাবোল
12  জীবনের সব দুঃখ ব্যথা আর \nনির্যাতনার কারণ সব ত...       কবিতা
18  এই শহরের কোন এক প্রেমিক চেয়েছিলো \nশীতের একলা ...  আবোল তাবোল


## Data Cleaning

In [2]:
# Removing empty contents
df = df[df.content.str.len() != 0 ]

In [3]:
# Unique Labels
labels = list(set(df.tags))
print(labels)

['গল্প', 'আবোল তাবোল', 'কবিতা', 'সমসাময়িক', 'কিছু একটা লিখতে ইচ্ছে হচ্ছে']


In [4]:
# Printing label contents
for label in labels:
    print( "{:20} --- {} samples".format( label, df[ df.tags == label ].shape[0]  ))

গল্প                 --- 898 samples
আবোল তাবোল           --- 447 samples
কবিতা                --- 3184 samples
সমসাময়িক             --- 1 samples
কিছু একটা লিখতে ইচ্ছে হচ্ছে --- 616 samples


In [5]:
# Mapping Labels to integer s
index2label = { i: labels[i] for i in range(len(labels)) }
label2index = { labels[i] : i for i in range(len(labels)) }

# Replacing labels to integers
df.tags = df.tags.replace(label2index)
print(df.head())

                                              content  tags
1   অফিসে তেমন ব্যস্ততা নেই, \nকেমন একটা নিঃসঙ্গ ভ...     2
8   সুখ আমি কল্পনা করব না \nআমার তো নেই সুখ এই প্র...     2
10  আমাদের ফেসবুক ঠিকানা: \nগ্রুপ: Facebook.com/gr...     1
12  জীবনের সব দুঃখ ব্যথা আর \nনির্যাতনার কারণ সব ত...     2
18  এই শহরের কোন এক প্রেমিক চেয়েছিলো \nশীতের একলা ...     1


In [6]:
# Stop Words that should be ignored
STOP_WORDS = list(set("""
অতএব অথচ অথবা অনুযায়ী অনেক অনেকে অনেকেই অন্তত  অবধি অবশ্য অর্থাৎ অন্য অনুযায়ী অর্ধভাগে
আগামী আগে আগেই আছে আজ আদ্যভাগে আপনার আপনি আবার আমরা আমাকে আমাদের আমার  আমি আর আরও 
ইত্যাদি ইহা 
উচিত উনি উপর উপরে উত্তর
এ এঁদের এঁরা এই এক একই একজন একটা একটি  একবার একে এখন এখনও এখানে এখানেই এটা এসো
এটাই এটি এত এতটাই এতে এদের এবং এবার এমন এমনি এমনকি এর এরা এলো এস এসে 
ঐ 
ও ওঁদের ওঁর ওঁরা ওই ওকে ওখানে ওদের ওর ওরা 
কখনও কত কথা কবে কয়েক  কয়েকটি করছে করছেন করতে  করবে করবেন করলে কয়েক  কয়েকটি করিয়ে করিয়া করায়
করলেন করা করাই করায় করার করি করিতে করিয়া করিয়ে করে করেই করেছিলেন করেছে করেছেন করেন কাউকে 
কাছ কাছে কাজ কাজে কারও কারণ কি কিংবা কিছু কিছুই কিন্তু কী কে কেউ কেউই কেন কোন কোনও কোনো কেমনে কোটি
ক্ষেত্রে খুব 
গিয়ে গিয়েছে গুলি গেছে গেল গেলে গোটা গিয়ে গিয়েছে
চলে চান চায় চেয়ে চায় চেয়ে চার চালু চেষ্টা 
ছাড়া ছাড়াও ছিল ছিলেন ছাড়া ছাড়াও
জন জনকে জনের জন্য জন্যে জানতে জানা জানানো জানায়  জানিয়ে  জানিয়েছে জানায় জাানিয়ে জানিয়েছে
টি 
ঠিক 
তখন তত তথা তবু তবে তা তাঁকে তাঁদের তাঁর তাঁরা তাঁহারা তাই তাও তাকে তাতে তাদের তার তারপর তারা তারই তাহলে তাহা তাহাতে তাহার তিনই 
তিনি তিনিও তুমি তুলে তেমন তো তোমার তুই তোরা তোর তোমাদের তোদের
থাকবে থাকবেন থাকা থাকায় থাকে থাকেন থেকে থেকেই  থেকেও থাকায়
দিকে দিতে দিয়ে দিয়েছে দিয়েছেন দিলেন দিয়ে দু  দুটি  দুটো দেওয়া দেওয়ার দেখতে দেখা দেখে দেন দেয়  দেশের  
দ্বারা দিয়েছে দিয়েছেন দেয় দেওয়া দেওয়ার দিন দুই
ধরা ধরে 
নয় না নাই নাকি নাগাদ নানা নিজে নিজেই নিজেদের নিজের নিতে নিয়ে নিয়ে নেই নেওয়া নেওয়ার নয় নতুন
পক্ষে পর পরে পরেই পরেও পর্যন্ত পাওয়া পারি পারে পারেন পেয়ে প্রতি প্রভৃতি প্রায় পাওয়া পেয়ে প্রায় পাঁচ প্রথম প্রাথমিক
ফলে ফিরে ফের 
বছর বদলে বরং বলতে বলল বললেন বলা বলে বলেছেন বলেন  বসে বহু বা বাদে বার বিনা বিভিন্ন বিশেষ বিষয়টি বেশ ব্যবহার ব্যাপারে বক্তব্য বন বেশি
ভাবে  ভাবেই 
মত মতো মতোই মধ্যভাগে মধ্যে মধ্যেই  মধ্যেও মনে মাত্র মাধ্যমে মানুষ মানুষের মোট মোটেই মোদের মোর 
যখন যত যতটা যথেষ্ট যদি যদিও যা যাঁর যাঁরা যাওয়া  যাওয়ার যাকে যাচ্ছে যাতে যাদের যান যাবে যায় যার  যারা যায় যিনি যে যেখানে যেতে যেন 
যেমন 
রকম রয়েছে রাখা রেখে রয়েছে 
লক্ষ 
শুধু শুরু 
সাধারণ সামনে সঙ্গে সঙ্গেও সব সবার সমস্ত সম্প্রতি সময় সহ সহিত সাথে সুতরাং সে  সেই সেখান সেখানে  সেটা সেটাই সেটাও সেটি স্পষ্ট স্বয়ং 
হইতে হইবে হইয়া হওয়া হওয়ায় হওয়ার হচ্ছে হত হতে হতেই হন হবে হবেন হয় হয়তো হয়নি হয়ে হয়েই হয়েছিল হয়েছে হাজার
হয়েছেন হল হলে হলেই হলেও হলো হিসাবে হিসেবে হৈলে হোক হয় হয়ে হয়েছে হৈতে হইয়া  হয়েছিল হয়েছেন হয়নি হয়েই হয়তো হওয়া হওয়ার হওয়ায়
""".split()))

In [7]:
# Sample content
sample = df.content.iloc[0]
print(sample)

অফিসে তেমন ব্যস্ততা নেই, 
কেমন একটা নিঃসঙ্গ ভাব 
মনটাকে পেতে আজকে আমার 
দিয়েছে বদলে আগের স্বভাব। 
ফেলে আসা দিন গুলো ভাবাচ্ছে, 
স্মৃতিরা আমাকে অতিষ্ঠ করে 
হয়তো বা কোন প্রতিশোধ নিতে- 
চাচ্ছে, ওরাও জুলম করছে। 
হয়ত ঘোরের মধ্যে পড়ে গেছি, 
একজন তুমি বা কাল্পনিক - 
প্রেমীকাকে খুব বিনয় করছি, 
বলছি, আমায় তুমিও বোঝ না? 
কেন কাছে আসো না ভালবাস না, 
তাহলে তো আর কিছুই হবে না, 
আমার কখনো - 
বিয়েই হবে না, সুখ মিলবে না । 
হা হা হা, দারুণ মজার তাই না? 
এমন মজার সময় কাটছে 
তবুও আমার ভাল লাগছে না, 
নিজেকে বন্দী বন্দী লাগছে 
সবকিছু যেন অচেনা অজানা। 
চেনা জানা সুখ গুলো নেই আর, 
কোথায় যে সব গিয়েছে হারিয়ে 
কেউ তার খোঁজ হয়ত জানে না 
ঠিকানা কোথায় বলতে পারে না। 
যৌবনে একাকী থাকা কষ্টের, 
একারণেই কি কাজে গতি আসে? 
সবাই চেষ্টা করে তারাতারি 
সঙ্গিনী আর সফলতা পেতে। 
আমার কি তবে প্রয়োজন সেই - 
সফলতা আর সেই সঙ্গিনী? 
হয়তো বা তাই, 
একারণেই তো বন্য হয়ে যাই। 
আমার বন্যতা হরেক রকম 
কান্না হাসির , পড়ার লেখার, 
বলার , চলার, দেবার নেবার 
ঘুমের, জাগার ইত্যাদি ইত্যাদি। 
এর সবি আমি প্রকাশ করেছি 
এর সবি হল আপন স্বভাব।


In [8]:
# Punctuations and Numeric Characters to ignore
filters = """
!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n?,।!‍.0123456789০১২৩৪৫৬৭৮৯
"""

# Remove all filtered characters
translate_dict = dict((c, ' ') for c in filters)
translate_map = str.maketrans(translate_dict)

# Applied to sample content
print(sample.translate(translate_map))

অফিসে তেমন ব্যস্ততা নেই   কেমন একটা নিঃসঙ্গ ভাব  মনটাকে পেতে আজকে আমার  দিয়েছে বদলে আগের স্বভাব   ফেলে আসা দিন গুলো ভাবাচ্ছে   স্মৃতিরা আমাকে অতিষ্ঠ করে  হয়তো বা কোন প্রতিশোধ নিতে   চাচ্ছে  ওরাও জুলম করছে   হয়ত ঘোরের মধ্যে পড়ে গেছি   একজন তুমি বা কাল্পনিক    প্রেমীকাকে খুব বিনয় করছি   বলছি  আমায় তুমিও বোঝ না   কেন কাছে আসো না ভালবাস না   তাহলে তো আর কিছুই হবে না   আমার কখনো    বিয়েই হবে না  সুখ মিলবে না    হা হা হা  দারুণ মজার তাই না   এমন মজার সময় কাটছে  তবুও আমার ভাল লাগছে না   নিজেকে বন্দী বন্দী লাগছে  সবকিছু যেন অচেনা অজানা   চেনা জানা সুখ গুলো নেই আর   কোথায় যে সব গিয়েছে হারিয়ে  কেউ তার খোঁজ হয়ত জানে না  ঠিকানা কোথায় বলতে পারে না   যৌবনে একাকী থাকা কষ্টের   একারণেই কি কাজে গতি আসে   সবাই চেষ্টা করে তারাতারি  সঙ্গিনী আর সফলতা পেতে   আমার কি তবে প্রয়োজন সেই    সফলতা আর সেই সঙ্গিনী   হয়তো বা তাই   একারণেই তো বন্য হয়ে যাই   আমার বন্যতা হরেক রকম  কান্না হাসির   পড়ার লেখার   বলার   চলার  দেবার নেবার  ঘুমের  জাগার ইত্যাদি ইত্যাদি   এর সবি আমি প্রকাশ করেছি  এর সবি হল আপন স্বভাব 


In [9]:
# Split by spaces
words = sample.translate(translate_map).split()
print(words)

['অফিসে', 'তেমন', 'ব্যস্ততা', 'নেই', 'কেমন', 'একটা', 'নিঃসঙ্গ', 'ভাব', 'মনটাকে', 'পেতে', 'আজকে', 'আমার', 'দিয়েছে', 'বদলে', 'আগের', 'স্বভাব', 'ফেলে', 'আসা', 'দিন', 'গুলো', 'ভাবাচ্ছে', 'স্মৃতিরা', 'আমাকে', 'অতিষ্ঠ', 'করে', 'হয়তো', 'বা', 'কোন', 'প্রতিশোধ', 'নিতে', 'চাচ্ছে', 'ওরাও', 'জুলম', 'করছে', 'হয়ত', 'ঘোরের', 'মধ্যে', 'পড়ে', 'গেছি', 'একজন', 'তুমি', 'বা', 'কাল্পনিক', 'প্রেমীকাকে', 'খুব', 'বিনয়', 'করছি', 'বলছি', 'আমায়', 'তুমিও', 'বোঝ', 'না', 'কেন', 'কাছে', 'আসো', 'না', 'ভালবাস', 'না', 'তাহলে', 'তো', 'আর', 'কিছুই', 'হবে', 'না', 'আমার', 'কখনো', 'বিয়েই', 'হবে', 'না', 'সুখ', 'মিলবে', 'না', 'হা', 'হা', 'হা', 'দারুণ', 'মজার', 'তাই', 'না', 'এমন', 'মজার', 'সময়', 'কাটছে', 'তবুও', 'আমার', 'ভাল', 'লাগছে', 'না', 'নিজেকে', 'বন্দী', 'বন্দী', 'লাগছে', 'সবকিছু', 'যেন', 'অচেনা', 'অজানা', 'চেনা', 'জানা', 'সুখ', 'গুলো', 'নেই', 'আর', 'কোথায়', 'যে', 'সব', 'গিয়েছে', 'হারিয়ে', 'কেউ', 'তার', 'খোঁজ', 'হয়ত', 'জানে', 'না', 'ঠিকানা', 'কোথায়', 'বলতে', 'পারে', 'না', 'যৌবনে', 'একাকী', 'থাকা', 'কষ্টের', 'একারণেই', 'কি

In [10]:
# Function to remove stopwords and filtered characters
def sentence_to_wordlist(sentence, filters="!\"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n?,।!‍.'0123456789০১২৩৪৫৬৭৮৯‘\u200c–“”…‘"):
    # just to be safe
    sentence = sentence.lower()
    translate_dict = dict((c, ' ') for c in filters)
    translate_map = str.maketrans(translate_dict)
    wordlist = sentence.translate(translate_map).split()
    return list(filter(lambda x: x not in STOP_WORDS, wordlist))

## Class for tokens

In [11]:
class Vocabulary(object):
    def __init__(self, documents=None):
        self.token2id = defaultdict(lambda : 0)
        self.id2token = defaultdict(lambda : ' ')
        # token count in a document
        self.dfs = {}
        self.token_collection = set()
        self.token_counter = Counter()
        self.documents = []
        
        if documents != None:
            self.update_documents(documents)
            
    def update_token_dictionary(self, remove_previous=False):
        
        if remove_previous:
            self.token2id = defaultdict(lambda : 0)
            self.id2token = defaultdict(lambda : ' ')
            self.token2id.update({' ' : 0})
            self.id2token.update({0 : ' '})
        
        self.id2token.update({ idx+1: word for idx, word in enumerate(self.token_collection) })
        self.token2id.update({ word: idx+1 for idx, word in enumerate(self.token_collection) })
        
            
    
    def update_documents(self, documents):
        # Extending the documents
        self.documents.extend(documents)
        
        for idx, document in enumerate(documents):
            for word in document:
                self.token_counter[word] += 1
                self.token_collection.add(word)
        
        self.update_token_dictionary()
        
        
    
    # add_documents removes the previous ones
    def add_documents(self, documents):
        self.documents = []
        self.token_counter = Counter()
        self.token2id = defaultdict(lambda : 0)
        self.id2token = defaultdict(lambda : ' ')
        self.update_documents(documents)
        
    def __len__(self):
        assert len(self.id2token) == len(self.token2id)
        return len(self.id2token)
    
    def get_token_count(self, token):
        if token not in self.token2id.keys():
            raise ValueError("Token doesn't exist")
        return self.token_counter[token]
    
    
    def reduce_vocabulary_by_frequency(self, threshold):
        token_collection = [ tok for tok in self.token_counter if self.token_counter[tok] > threshold ]
        self.token_collection = set(token_collection)
        self.update_token_dictionary(remove_previous=True)
        
    def bow_from_saved_documents(self):
        bow_matrix = np.zeros((len(self.documents), len(self.token_collection)))
        
        for i, doc in enumerate(self.documents):
            for tok in doc:
                bow_matrix[i, self.token2id[tok] - 1] += 1
        
        return bow_matrix
    
    def doc2bow(self, doc):
        bow = np.zeros(len(self.token_collection) + 1)
        
        for tok in doc:
            bow[v.token2id[tok]] += 1
            
        return bow[1:]
    
    
    def __getitem__(self, key):
        if key >= len(self.id2token):
            raise KeyError("Key can't be equal or greater than total token count")
        return self.id2token[key]

In [12]:
# Create documents for vocabulary class
documents = [ sentence_to_wordlist(sent) for sent in tqdm(df.content)  ]

100%|█████████████████████████████████████████████████████████████████████████████| 5146/5146 [00:20<00:00, 251.71it/s]


## Training Model (using Logistic Regression)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [14]:
# Discard tokens below this frequency threshold
FREQ_THRESHOLD = 50
v = Vocabulary()
v.add_documents(documents)
v.reduce_vocabulary_by_frequency(FREQ_THRESHOLD)

In [15]:
# Train test data 
train_x = v.bow_from_saved_documents()
train_y = np.asarray(df.tags)

train_x, train_y = shuffle(train_x, train_y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.33, random_state=42)
lr = LogisticRegression(C=1e5,max_iter=1e5)
lr.fit(X_train, y_train)

LogisticRegression(C=100000.0, max_iter=100000.0)

In [16]:
# Get predictions
y_pred = lr.predict(X_test)

In [17]:
# Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.50      0.69      0.58       297
           1       0.25      0.24      0.25       139
           2       0.84      0.77      0.80      1076
           4       0.23      0.19      0.20       187

    accuracy                           0.65      1699
   macro avg       0.45      0.47      0.46      1699
weighted avg       0.66      0.65      0.65      1699



## Test

In [18]:
test_string = """
  এসো, এসো, এসো হে বৈশাখ
তাপসনিশ্বাসবায়ে মুমূর্ষুরে দাও উড়ায়ে,
বৎসরের আবর্জনা দূর হয়ে যাক
যাক পুরাতন স্মৃতি, যাক ভুলে যাওয়া গীতি,
অশ্রুবাষ্প সুদূরে মিলাক।
মুছে যাক গ্লানি, ঘুচে যাক জরা,
অগ্নিস্নানে শুচি হোক ধরা
রসের আবেশরাশি শুষ্ক করি দাও আসি,
আনো আনো আনো তব প্রলয়ের শাঁখ
মায়ার কুজ্ঝটিজাল যাক দূরে যাক।
  """

doc = sentence_to_wordlist(test_string)
bow_of_doc = v.doc2bow(doc)
output = lr.predict([bow_of_doc])
print(index2label[output[0]])

কবিতা
